In [10]:
import requests
import base64
import dotenv
import os
import uuid
import jwt 
from pprint import pprint

def get_access_token(client_id, client_secret, scope="default_scope"):
    url = "https://portal.api.bolagsverket.se/oauth2/token"
    credentials = f"{client_id}:{client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "scope": "vardefulla-datamangder:read vardefulla-datamangder:ping"  # ✅ Correct scopes
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        print(response.text)
        return response.json()["access_token"], response.json().get("expires_in", 3600)
    else:
        raise Exception(f"Failed to get access token: {response.text}")

def revoke_access_token(client_id, client_secret, token):
    url = "https://portal.api.bolagsverket.se/oauth2/revoke"
    credentials = f"{client_id}:{client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"token": token}
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        print("Token revoked successfully")
    else:
        raise Exception(f"Failed to revoke token: {response.text}")
    
def load_dotenv():
    dotenv.load_dotenv()
    client_id = os.getenv("BOLAGSVARKET_CLIENT_ID")
    client_secret = os.getenv("BOLAGSVARKET_CLIENT_SECRET")
    return client_id, client_secret

def api_request(endpoint, token, payload=None):
    url = f"https://gw.api.bolagsverket.se/vardefulla-datamangder/v1/{endpoint}"
    request_id = str(uuid.uuid4())
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
        "X-Request-Id": request_id
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data from {endpoint}: {response.text}")

def is_alive_api_request(token):
    url = "https://gw.api.bolagsverket.se/vardefulla-datamangder/v1/isalive"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API is not alive: {response.text}")
    
    
    

In [7]:
client_id, client_secret = load_dotenv()
access_token, expires_in = get_access_token(client_id, client_secret)
print(f"Access token: {access_token}")



decoded_token = jwt.decode(access_token, options={"verify_signature": False})
print(f"Decoded token: {decoded_token}")


{"access_token":"eyJ4NXQiOiJOVGRtWmpNNFpEazNOalkwWXpjNU1tWm1PRGd3TVRFM01XWXdOREU1TVdSbFpEZzROemM0WkEiLCJraWQiOiJNell4TW1Ga09HWXdNV0kwWldObU5EY3hOR1l3WW1NNFpUQTNNV0kyTkRBelpHUXpOR00wWkdSbE5qSmtPREZrWkRSaU9URmtNV0ZoTXpVMlpHVmxOZ19SUzI1NiIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJwaGlsaXByZXR0aWcxMUBnbWFpbC5jb20iLCJhdXQiOiJBUFBMSUNBVElPTiIsImF1ZCI6InFqU091MkVZRm5YSUlLY3AwOU1BaXpUZmtCY2EiLCJuYmYiOjE3NDE5ODQzNDQsIm9yZ25hbWUiOiJWw6RyZGVmdWxsYSBkYXRhbcOkbmdkZXIiLCJhenAiOiJxalNPdTJFWUZuWElJS2NwMDlNQWl6VGZrQmNhIiwic2NvcGUiOiJ2YXJkZWZ1bGxhLWRhdGFtYW5nZGVyOnBpbmcgdmFyZGVmdWxsYS1kYXRhbWFuZ2RlcjpyZWFkIiwiaXNzIjoiaHR0cHM6XC9cL3BvcnRhbC5hcGkuYm9sYWdzdmVya2V0LnNlXC9vYXV0aDJcL3Rva2VuIiwib3JnbnIiOiI5OTk5OTk5OTk5IiwiZXhwIjoxNzQxOTg3OTQ0LCJpYXQiOjE3NDE5ODQzNDQsImp0aSI6ImZlN2Y3OGMwLTliZWUtNDJhMi1hYTFjLWIxZGU2YzUzZjQ0NSJ9.Zt52P7unQCakUfjKXgHBbyjZSUmEAuAc5ACGfqoJ5i65gz4k4Ugijx9WAQsOcIAnTkHIzO8acR6e35UJR-vJwdHb6rE9xYkIfvfer3fBaytS6RyVVZGBvYoGfFU0EXVknpAk7Ii68ZGj6PrlOtsZpVlIJgzhGuh8VF9N9WE3ZhHUxg7B-rCGA-uDlNHhoi19Q2wgCC

In [18]:
valid_endpoints = [ "organisationer", "isalive"]
org_data = api_request("organisationer", access_token, {"identitetsbeteckning": "5595019059"})
pprint(org_data)

 

{'organisationer': [{'avregistreradOrganisation': None,
                     'avregistreringsorsak': None,
                     'juridiskForm': {'dataproducent': 'SCB',
                                      'fel': None,
                                      'klartext': 'Övriga aktiebolag',
                                      'kod': '49'},
                     'namnskyddslopnummer': None,
                     'naringsgrenOrganisation': {'dataproducent': 'SCB',
                                                 'fel': None,
                                                 'sni': [{'klartext': 'Dataprogrammering',
                                                          'kod': '62010'},
                                                         {'klartext': '',
                                                          'kod': '     '},
                                                         {'klartext': '',
                                                          'kod': '     '},
        